<a href="https://colab.research.google.com/github/ghommidhWassim/GNN-variants/blob/main/FastGCN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git
!python -c "import torch; print(torch.__version__)"
!python -c "import torch; print(torch.version.cuda)"
!pip install torchvision
!pip install pyg_lib torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-2.6.0+cu124.html


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
2.6.0+cu124
12.4
Looking in links: https://data.pyg.org/whl/torch-2.6.0+cu124.html


In [2]:
import torch
import typing
import math
import numpy as np
import torch.nn as nn
import scipy.sparse as sp
import torch_geometric.utils as utils
import time
# Try micro F1 score
from sklearn.metrics import f1_score as F1
from torch_geometric.datasets import Planetoid, Amazon
from torch_geometric.utils import to_scipy_sparse_matrix
from torch_geometric.transforms import NormalizeFeatures, RandomNodeSplit
import torch_geometric.transforms as T
from torch_sparse import SparseTensor
import torch.nn.functional as F
import json
from sklearn.metrics import f1_score


In [3]:
def clean_gpu_memory():
    """Cleans GPU memory without fully resetting the CUDA context"""
    import gc
    gc.collect()  # Python garbage collection
    if torch.cuda.is_available():
        torch.cuda.empty_cache()  # PyTorch cache
        torch.cuda.reset_peak_memory_stats()  # Reset tracking
        print(f"Memory after cleanup: {torch.cuda.memory_allocated()/1024**2:.2f} MB")


In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def dataset_load():
  print(f"Using device: {device}")
  dataset = Planetoid(root='data/Planetoid', name='PubMed', transform=NormalizeFeatures())
  num_features = dataset.num_features
  num_classes = dataset.num_classes
  data = dataset[0].to(device)  # Get the first graph object.
  return num_features, data, num_classes, device,dataset
num_features, data, num_classes, device, dataset = dataset_load()
print(f'Number of nodes:          {data.num_nodes}')
print(f'Number of edges:          {data.num_edges}')
print(f'Average node degree:      {data.num_edges / data.num_nodes:.2f}')
print(f'Number of training nodes: {data.train_mask.sum()}')
print(f'Training node label rate: {int(data.train_mask.sum()) / data.num_nodes:.3f}')
print(f'Has isolated nodes:       {data.has_isolated_nodes()}')
print(f'Has self-loops:           {data.has_self_loops()}')
print(f'Is undirected:            {data.is_undirected()}')
edges = data.edge_index  # Edge indices [2, num_edges]
labels = data.y          # Node labels [num_nodes]
feat_data = data.x       # Node features [num_nodes, num_features]
num_nodes = data.num_nodes  # This is already an integer value
num_classes = dataset.num_classes
num_features = dataset.num_features
train_nodes = torch.where(data.train_mask)[0]
valid_nodes = torch.where(data.val_mask)[0]
test_nodes = torch.where(data.test_mask)[0]

Using device: cuda
Number of nodes:          19717
Number of edges:          88648
Average node degree:      4.50
Number of training nodes: 60
Training node label rate: 0.003
Has isolated nodes:       False
Has self-loops:           False
Is undirected:            True


In [5]:
adj = SparseTensor(
    row=data.edge_index[0],
    col=data.edge_index[1],
    value=torch.ones(data.edge_index.size(1), device=device),
    sparse_sizes=(data.num_nodes, data.num_nodes)
)

# For normalized Laplacian (GPU)
deg = adj.sum(dim=1).pow(-0.5)
adj_norm = deg.view(-1, 1) * adj * deg.view(1, -1)  # Normalized adjacency

In [6]:
def row_normalize_sparse_tensor(adj):
    row_sum = adj.sum(dim=1)
    row_sum_inv = row_sum.pow(-1)
    row_sum_inv[row_sum_inv == float('inf')] = 0

    n = adj.size(0)
    norm = SparseTensor(
        row=torch.arange(n, device=device),
        col=torch.arange(n, device=device),
        value=row_sum_inv,
        sparse_sizes=(n, n)
    )
    return norm @ adj

# Create normalized Laplacian
eye = SparseTensor.eye(data.num_nodes, device=device)
lap_matrix = row_normalize_sparse_tensor(adj + eye)

feat_data = data.x if data.x.is_sparse else data.x
labels = data.y

In [7]:
def evaluate(model, features, adj_list, labels, nodes):
    model.eval()
    with torch.no_grad():
        out = model(features, adj_list)
        pred = out[nodes].argmax(dim=1).cpu()
        true = labels[nodes].cpu()

        acc = (pred == true).float().mean().item()
        f1_micro = f1_score(true, pred, average='micro')

    return acc, f1_micro


def fastgcn_sampler(batch_nodes, samp_num_list, lap_matrix, depth, device):
    """
    GPU-optimized FastGCN sampler with fixed dimension handling

    Args:
        batch_nodes: Tensor of starting nodes [batch_size]
        samp_num_list: List of sample sizes per layer
        lap_matrix: SparseTensor of normalized Laplacian
        depth: Number of GCN layers
        device: Target device
    """
    previous_nodes = batch_nodes
    adjs = []

    # Precompute degree and probabilities
    deg = lap_matrix.sum(dim=1)
    p = deg / deg.sum()

    for d in range(depth):
        # Sample nodes
        s_num = min(int(torch.sum(p > 0)), samp_num_list[d])
        after_nodes = torch.multinomial(p, s_num, replacement=False)

        # Create subgraph adjacency
        adj = lap_matrix[previous_nodes][:, after_nodes]

        # Importance weighting - fix dimension handling
        p_sampled = p[after_nodes].view(1, -1)  # Shape [1, s_num]
        adj = adj / p_sampled  # Broadcasts correctly

        # Row normalize
        row_sum = adj.sum(dim=1).view(-1, 1)  # Shape [batch_size, 1]
        row_sum[row_sum == 0] = 1  # Prevent division by zero
        adj = adj / row_sum

        adjs.append(adj)
        previous_nodes = after_nodes

    return adjs[::-1], previous_nodes, batch_nodes

class GCN(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers):
        super().__init__()
        self.convs = nn.ModuleList()
        self.convs.append(nn.Linear(in_channels, hidden_channels))
        for _ in range(num_layers - 2):
            self.convs.append(nn.Linear(hidden_channels, hidden_channels))
        self.convs.append(nn.Linear(hidden_channels, out_channels))

    def forward(self, x, adjs):
        """
        Args:
            x: Node features [num_nodes, num_features]
            adjs: List of sampled adjacency matrices from fastgcn_sampler
        """
        for i, (conv, adj) in enumerate(zip(self.convs[:-1], adjs)):
            x = conv(x)
            x = adj @ x  # Sparse matrix multiplication
            x = F.relu(x)
        x = self.convs[-1](x)
        return x

# Initialize
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GCN(num_features, 16, num_classes, num_layers=2).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

# Sample parameters
samp_num_list = [64, 64]  # Sample sizes per layer
batch_size = 128
start_time = time.time()

for epoch in range(1,101):
    model.train()

    # Sample a batch - fixed indexing
    idx = torch.randperm(len(train_nodes), device=device)[:batch_size]
    batch_nodes = train_nodes[idx]

    adjs, sampled_nodes, _ = fastgcn_sampler(
        batch_nodes=batch_nodes,
        samp_num_list=samp_num_list,
        lap_matrix=lap_matrix,
        depth=2,
        device=device
    )

    # Forward pass - ensure we use sampled nodes
    optimizer.zero_grad()
    out = model(feat_data[sampled_nodes], adjs)
    loss = criterion(out, labels[sampled_nodes])

    # Backward pass
    loss.backward()
    optimizer.step()

    # Validation
    if epoch % 10 == 0:
        model.eval()
        with torch.no_grad():
            # Create full graph adjacencies for validation
            full_adjs = [lap_matrix] * 2
            val_out = model(feat_data, full_adjs)
        val_acc, val_f1 = evaluate(model, feat_data, [lap_matrix]*2, labels, valid_nodes)
        print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Val Acc: {val_acc:.4f}, F1 (micro): {val_f1:.4f}')

end_time = time.time()
test_acc, f1_micro = evaluate(model, feat_data, [lap_matrix]*2, labels, test_nodes)
print(f'Test Accuracy: {test_acc:.4f}, F1 (micro): {f1_micro:.4f}')

print(f"Training time: {end_time - start_time:.2f} seconds")
print(f"Current GPU memory: {torch.cuda.memory_allocated()/1024**2:.2f} MB")
print(f"Max GPU memory used: {torch.cuda.max_memory_allocated()/1024**2:.2f} MB")


Epoch: 010, Loss: 1.0596, Val Acc: 0.3880, F1 (micro): 0.3880
Epoch: 020, Loss: 1.0686, Val Acc: 0.3880, F1 (micro): 0.3880
Epoch: 030, Loss: 1.0226, Val Acc: 0.4160, F1 (micro): 0.4160
Epoch: 040, Loss: 1.0594, Val Acc: 0.4160, F1 (micro): 0.4160
Epoch: 050, Loss: 0.9906, Val Acc: 0.4160, F1 (micro): 0.4160
Epoch: 060, Loss: 1.0476, Val Acc: 0.4160, F1 (micro): 0.4160
Epoch: 070, Loss: 1.0873, Val Acc: 0.4160, F1 (micro): 0.4160
Epoch: 080, Loss: 1.0440, Val Acc: 0.4160, F1 (micro): 0.4160
Epoch: 090, Loss: 1.1390, Val Acc: 0.4160, F1 (micro): 0.4160
Epoch: 100, Loss: 1.0266, Val Acc: 0.4160, F1 (micro): 0.4160
Test Accuracy: 0.4070, F1 (micro): 0.4070
Training time: 1.41 seconds
Current GPU memory: 61.19 MB
Max GPU memory used: 64.50 MB


In [8]:
def estimate_fastgcn_memory_MB(K, L, slayer, b=None):
    """
    Estimate GPU memory usage for FastGCN embedding + transformation weights.

    Args:
        K (int): Hidden/embedding dimension per layer
        L (int): Number of GCN layers
        slayer (int): Number of sampled nodes per layer
        b (int, optional): Batch size. If None, assume b = slayer

    Returns:
        memory_MB (float): Estimated memory in megabytes (MB)
    """
    if b is None:
        b = slayer  # worst-case assumption as per LADIES paper

    # Main memory formula from LADIES paper approximation
    total_floats = L * K * slayer + L * K * K
    total_bytes = total_floats * 4  # float32 = 4 bytes
    memory_MB = total_bytes / (1024 ** 2)

    return memory_MB


K = 16          # hidden dimension
L = 2            # number of layers
slayer = 64    # number of sampled nodes per layer
b = 128          # batch size (optional)

mem_MB = estimate_fastgcn_memory_MB(K, L, slayer, b)
print(f"Estimated FastGCN memory usage: {mem_MB:.2f} MB")


Estimated FastGCN memory usage: 0.01 MB


In [9]:
peak_memory_mb=f"{torch.cuda.max_memory_allocated()/1024**2:.2f}"
total_train_time=f"{end_time - start_time:.2f}"

metrics = {
    "model": "graphSAGE",
    "accuracy": test_acc,
    "f1_micro":f1_micro,
    "peak_memory_MB": peak_memory_mb,
    "train_time_sec": total_train_time,
    "mem_MB":mem_MB
}

with open("fastGCN_pubmed_results.json", "w") as f:
    json.dump(metrics, f)

**CORA Dataset**

In [10]:
clean_gpu_memory()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def dataset_load():
  print(f"Using device: {device}")
  dataset = Planetoid(root='data/Planetoid', name='Cora', transform=NormalizeFeatures())
  num_features = dataset.num_features
  num_classes = dataset.num_classes
  data = dataset[0].to(device)  # Get the first graph object.
  return num_features, data, num_classes, device,dataset
num_features, data, num_classes, device, dataset = dataset_load()
print(f'Number of nodes:          {data.num_nodes}')
print(f'Number of edges:          {data.num_edges}')
print(f'Average node degree:      {data.num_edges / data.num_nodes:.2f}')
print(f'Number of training nodes: {data.train_mask.sum()}')
print(f'Training node label rate: {int(data.train_mask.sum()) / data.num_nodes:.3f}')
print(f'Has isolated nodes:       {data.has_isolated_nodes()}')
print(f'Has self-loops:           {data.has_self_loops()}')
print(f'Is undirected:            {data.is_undirected()}')
edges = data.edge_index  # Edge indices [2, num_edges]
labels = data.y          # Node labels [num_nodes]
feat_data = data.x       # Node features [num_nodes, num_features]
num_nodes = data.num_nodes  # This is already an integer value
num_classes = dataset.num_classes
num_features = dataset.num_features
train_nodes = torch.where(data.train_mask)[0]
valid_nodes = torch.where(data.val_mask)[0]
test_nodes = torch.where(data.test_mask)[0]


adj = SparseTensor(
    row=data.edge_index[0],
    col=data.edge_index[1],
    value=torch.ones(data.edge_index.size(1), device=device),
    sparse_sizes=(data.num_nodes, data.num_nodes)
)

# For normalized Laplacian (GPU)
deg = adj.sum(dim=1).pow(-0.5)
adj_norm = deg.view(-1, 1) * adj * deg.view(1, -1)  # Normalized adjacency

eye = SparseTensor.eye(data.num_nodes, device=device)
lap_matrix = row_normalize_sparse_tensor(adj + eye)

feat_data = data.x if data.x.is_sparse else data.x
labels = data.y

# Initialize
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GCN(num_features, 16, num_classes, num_layers=2).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

# Sample parameters
samp_num_list = [64, 64]  # Sample sizes per layer
batch_size = 128
start_time = time.time()

for epoch in range(1,101):
    model.train()

    # Sample a batch - fixed indexing
    idx = torch.randperm(len(train_nodes), device=device)[:batch_size]
    batch_nodes = train_nodes[idx]

    adjs, sampled_nodes, _ = fastgcn_sampler(
        batch_nodes=batch_nodes,
        samp_num_list=samp_num_list,
        lap_matrix=lap_matrix,
        depth=2,
        device=device
    )

    # Forward pass - ensure we use sampled nodes
    optimizer.zero_grad()
    out = model(feat_data[sampled_nodes], adjs)
    loss = criterion(out, labels[sampled_nodes])

    # Backward pass
    loss.backward()
    optimizer.step()

    # Validation
    if epoch % 10 == 0:
        model.eval()
        with torch.no_grad():
            # Create full graph adjacencies for validation
            full_adjs = [lap_matrix] * 2
            val_out = model(feat_data, full_adjs)
        val_acc, val_f1 = evaluate(model, feat_data, [lap_matrix]*2, labels, valid_nodes)
        print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Val Acc: {val_acc:.4f}, F1 (micro): {val_f1:.4f}')

end_time = time.time()
test_acc, f1_micro = evaluate(model, feat_data, [lap_matrix]*2, labels, test_nodes)
print(f'Test Accuracy: {test_acc:.4f}, F1 (micro): {f1_micro:.4f}')

print(f"Training time: {end_time - start_time:.2f} seconds")
print(f"Current GPU memory: {torch.cuda.memory_allocated()/1024**2:.2f} MB")
print(f"Max GPU memory used: {torch.cuda.max_memory_allocated()/1024**2:.2f} MB")

Memory after cleanup: 61.19 MB
Using device: cuda
Number of nodes:          2708
Number of edges:          10556
Average node degree:      3.90
Number of training nodes: 140
Training node label rate: 0.052
Has isolated nodes:       False
Has self-loops:           False
Is undirected:            True
Epoch: 010, Loss: 1.8997, Val Acc: 0.3160, F1 (micro): 0.3160
Epoch: 020, Loss: 1.9192, Val Acc: 0.3160, F1 (micro): 0.3160
Epoch: 030, Loss: 1.8926, Val Acc: 0.3160, F1 (micro): 0.3160
Epoch: 040, Loss: 1.8204, Val Acc: 0.3160, F1 (micro): 0.3160
Epoch: 050, Loss: 1.7991, Val Acc: 0.3160, F1 (micro): 0.3160
Epoch: 060, Loss: 1.8465, Val Acc: 0.3160, F1 (micro): 0.3160
Epoch: 070, Loss: 1.7944, Val Acc: 0.3160, F1 (micro): 0.3160
Epoch: 080, Loss: 1.8856, Val Acc: 0.3160, F1 (micro): 0.3160
Epoch: 090, Loss: 1.9360, Val Acc: 0.3160, F1 (micro): 0.3160
Epoch: 100, Loss: 1.8957, Val Acc: 0.3160, F1 (micro): 0.3160
Test Accuracy: 0.3190, F1 (micro): 0.3190
Training time: 2.12 seconds
Current G

In [11]:
peak_memory_mb=f"{torch.cuda.max_memory_allocated()/1024**2:.2f}"
total_train_time=f"{end_time - start_time:.2f}"

metrics = {
    "model": "graphSAGE",
    "accuracy": test_acc,
    "f1_micro":f1_micro,
    "peak_memory_MB": peak_memory_mb,
    "train_time_sec": total_train_time,
    "mem_MB":mem_MB
}

with open("fastGCN_Cora_results.json", "w") as f:
    json.dump(metrics, f)

**citeeser DATASET**

In [12]:
clean_gpu_memory()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def dataset_load():
  print(f"Using device: {device}")
  dataset = Planetoid(root='data/Planetoid', name='CiteSeer', transform=NormalizeFeatures())
  num_features = dataset.num_features
  num_classes = dataset.num_classes
  data = dataset[0].to(device)  # Get the first graph object.
  return num_features, data, num_classes, device,dataset
num_features, data, num_classes, device, dataset = dataset_load()
print(f'Number of nodes:          {data.num_nodes}')
print(f'Number of edges:          {data.num_edges}')
print(f'Average node degree:      {data.num_edges / data.num_nodes:.2f}')
print(f'Number of training nodes: {data.train_mask.sum()}')
print(f'Training node label rate: {int(data.train_mask.sum()) / data.num_nodes:.3f}')
print(f'Has isolated nodes:       {data.has_isolated_nodes()}')
print(f'Has self-loops:           {data.has_self_loops()}')
print(f'Is undirected:            {data.is_undirected()}')
edges = data.edge_index  # Edge indices [2, num_edges]
labels = data.y          # Node labels [num_nodes]
feat_data = data.x       # Node features [num_nodes, num_features]
num_nodes = data.num_nodes  # This is already an integer value
num_classes = dataset.num_classes
num_features = dataset.num_features
train_nodes = torch.where(data.train_mask)[0]
valid_nodes = torch.where(data.val_mask)[0]
test_nodes = torch.where(data.test_mask)[0]


adj = SparseTensor(
    row=data.edge_index[0],
    col=data.edge_index[1],
    value=torch.ones(data.edge_index.size(1), device=device),
    sparse_sizes=(data.num_nodes, data.num_nodes)
)

# For normalized Laplacian (GPU)
deg = adj.sum(dim=1).pow(-0.5)
adj_norm = deg.view(-1, 1) * adj * deg.view(1, -1)  # Normalized adjacency

eye = SparseTensor.eye(data.num_nodes, device=device)
lap_matrix = row_normalize_sparse_tensor(adj + eye)

feat_data = data.x if data.x.is_sparse else data.x
labels = data.y

# Initialize
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GCN(num_features, 16, num_classes, num_layers=2).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

# Sample parameters
samp_num_list = [64, 64]  # Sample sizes per layer
batch_size = 128
start_time = time.time()

for epoch in range(1,101):
    model.train()

    # Sample a batch - fixed indexing
    idx = torch.randperm(len(train_nodes), device=device)[:batch_size]
    batch_nodes = train_nodes[idx]

    adjs, sampled_nodes, _ = fastgcn_sampler(
        batch_nodes=batch_nodes,
        samp_num_list=samp_num_list,
        lap_matrix=lap_matrix,
        depth=2,
        device=device
    )

    # Forward pass - ensure we use sampled nodes
    optimizer.zero_grad()
    out = model(feat_data[sampled_nodes], adjs)
    loss = criterion(out, labels[sampled_nodes])

    # Backward pass
    loss.backward()
    optimizer.step()

    # Validation
    if epoch % 10 == 0:
        model.eval()
        with torch.no_grad():
            # Create full graph adjacencies for validation
            full_adjs = [lap_matrix] * 2
            val_out = model(feat_data, full_adjs)
        val_acc, val_f1 = evaluate(model, feat_data, [lap_matrix]*2, labels, valid_nodes)
        print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Val Acc: {val_acc:.4f}, F1 (micro): {val_f1:.4f}')

end_time = time.time()
test_acc, f1_micro = evaluate(model, feat_data, [lap_matrix]*2, labels, test_nodes)
print(f'Test Accuracy: {test_acc:.4f}, F1 (micro): {f1_micro:.4f}')

print(f"Training time: {end_time - start_time:.2f} seconds")
print(f"Current GPU memory: {torch.cuda.memory_allocated()/1024**2:.2f} MB")
print(f"Max GPU memory used: {torch.cuda.max_memory_allocated()/1024**2:.2f} MB")

Memory after cleanup: 74.15 MB
Using device: cuda
Number of nodes:          3327
Number of edges:          9104
Average node degree:      2.74
Number of training nodes: 120
Training node label rate: 0.036
Has isolated nodes:       True
Has self-loops:           False
Is undirected:            True
Epoch: 010, Loss: 1.7690, Val Acc: 0.1880, F1 (micro): 0.1880
Epoch: 020, Loss: 1.8027, Val Acc: 0.1880, F1 (micro): 0.1880
Epoch: 030, Loss: 1.7502, Val Acc: 0.1880, F1 (micro): 0.1880
Epoch: 040, Loss: 1.7831, Val Acc: 0.1740, F1 (micro): 0.1740
Epoch: 050, Loss: 1.7597, Val Acc: 0.2120, F1 (micro): 0.2120
Epoch: 060, Loss: 1.7965, Val Acc: 0.2120, F1 (micro): 0.2120
Epoch: 070, Loss: 1.7261, Val Acc: 0.2120, F1 (micro): 0.2120
Epoch: 080, Loss: 1.8047, Val Acc: 0.2120, F1 (micro): 0.2120
Epoch: 090, Loss: 1.7373, Val Acc: 0.2120, F1 (micro): 0.2120
Epoch: 100, Loss: 1.7267, Val Acc: 0.2120, F1 (micro): 0.2120
Test Accuracy: 0.2310, F1 (micro): 0.2310
Training time: 1.54 seconds
Current GPU

In [13]:
peak_memory_mb=f"{torch.cuda.max_memory_allocated()/1024**2:.2f}"
total_train_time=f"{end_time - start_time:.2f}"

metrics = {
    "model": "graphSAGE",
    "accuracy": test_acc,
    "f1_micro":f1_micro,
    "peak_memory_MB": peak_memory_mb,
    "train_time_sec": total_train_time,
    "mem_MB":mem_MB
}

with open("fastGCN_Citeser_results.json", "w") as f:
    json.dump(metrics, f)

**amazon**

In [14]:
clean_gpu_memory()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def dataset_load():
  print(f"Using device: {device}")
  dataset = Amazon(
        root='data/Amazon',
        name='Computers',
        transform=T.Compose([
        NormalizeFeatures(),          # feature‑wise ℓ₂ normalisation
        RandomNodeSplit(              # ⇦ add a split transform
                split='train_rest',       # 10% val, 10% test by default
                num_val=0.1,
                num_test=0.1,
                num_splits=1,
            )
        ])
    )
  num_features = dataset.num_features
  num_classes = dataset.num_classes
  data = dataset[0].to(device)  # Get the first graph object.
  return num_features, data, num_classes, device,dataset

num_features, data, num_classes, device, dataset = dataset_load()
print(f'Number of nodes:          {data.num_nodes}')
print(f'Number of edges:          {data.num_edges}')
print(f'Average node degree:      {data.num_edges / data.num_nodes:.2f}')
print(f'Number of training nodes: {data.train_mask.sum()}')
print(f'Training node label rate: {int(data.train_mask.sum()) / data.num_nodes:.3f}')
print(f'Has isolated nodes:       {data.has_isolated_nodes()}')
print(f'Has self-loops:           {data.has_self_loops()}')
print(f'Is undirected:            {data.is_undirected()}')
edges = data.edge_index  # Edge indices [2, num_edges]
labels = data.y          # Node labels [num_nodes]
feat_data = data.x       # Node features [num_nodes, num_features]
num_nodes = data.num_nodes  # This is already an integer value
num_classes = dataset.num_classes
num_features = dataset.num_features
train_nodes = torch.where(data.train_mask)[0]
valid_nodes = torch.where(data.val_mask)[0]
test_nodes = torch.where(data.test_mask)[0]


adj = SparseTensor(
    row=data.edge_index[0],
    col=data.edge_index[1],
    value=torch.ones(data.edge_index.size(1), device=device),
    sparse_sizes=(data.num_nodes, data.num_nodes)
)

# For normalized Laplacian (GPU)
deg = adj.sum(dim=1).pow(-0.5)
adj_norm = deg.view(-1, 1) * adj * deg.view(1, -1)  # Normalized adjacency

eye = SparseTensor.eye(data.num_nodes, device=device)
lap_matrix = row_normalize_sparse_tensor(adj + eye)

feat_data = data.x if data.x.is_sparse else data.x
labels = data.y

# Initialize
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GCN(num_features, 16, num_classes, num_layers=2).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

# Sample parameters
samp_num_list = [64, 64]  # Sample sizes per layer
batch_size = 128
start_time = time.time()

for epoch in range(1,101):
    model.train()

    # Sample a batch - fixed indexing
    idx = torch.randperm(len(train_nodes), device=device)[:batch_size]
    batch_nodes = train_nodes[idx]

    adjs, sampled_nodes, _ = fastgcn_sampler(
        batch_nodes=batch_nodes,
        samp_num_list=samp_num_list,
        lap_matrix=lap_matrix,
        depth=2,
        device=device
    )

    # Forward pass - ensure we use sampled nodes
    optimizer.zero_grad()
    out = model(feat_data[sampled_nodes], adjs)
    loss = criterion(out, labels[sampled_nodes])

    # Backward pass
    loss.backward()
    optimizer.step()

    # Validation
    if epoch % 10 == 0:
        model.eval()
        with torch.no_grad():
            # Create full graph adjacencies for validation
            full_adjs = [lap_matrix] * 2
            val_out = model(feat_data, full_adjs)
        val_acc, val_f1 = evaluate(model, feat_data, [lap_matrix]*2, labels, valid_nodes)
        print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Val Acc: {val_acc:.4f}, F1 (micro): {val_f1:.4f}')

end_time = time.time()
test_acc, f1_micro = evaluate(model, feat_data, [lap_matrix]*2, labels, test_nodes)
print(f'Test Accuracy: {test_acc:.4f}, F1 (micro): {f1_micro:.4f}')

print(f"Training time: {end_time - start_time:.2f} seconds")
print(f"Current GPU memory: {torch.cuda.memory_allocated()/1024**2:.2f} MB")
print(f"Max GPU memory used: {torch.cuda.max_memory_allocated()/1024**2:.2f} MB")

Memory after cleanup: 65.03 MB
Using device: cuda
Number of nodes:          13752
Number of edges:          491722
Average node degree:      35.76
Number of training nodes: 11002
Training node label rate: 0.800
Has isolated nodes:       True
Has self-loops:           False
Is undirected:            True
Epoch: 010, Loss: 2.2551, Val Acc: 0.3840, F1 (micro): 0.3840
Epoch: 020, Loss: 2.1644, Val Acc: 0.3840, F1 (micro): 0.3840
Epoch: 030, Loss: 2.1347, Val Acc: 0.3840, F1 (micro): 0.3840
Epoch: 040, Loss: 2.0510, Val Acc: 0.3840, F1 (micro): 0.3840
Epoch: 050, Loss: 1.9469, Val Acc: 0.3840, F1 (micro): 0.3840
Epoch: 060, Loss: 2.0157, Val Acc: 0.3840, F1 (micro): 0.3840
Epoch: 070, Loss: 1.8924, Val Acc: 0.3840, F1 (micro): 0.3840
Epoch: 080, Loss: 2.0311, Val Acc: 0.3840, F1 (micro): 0.3840
Epoch: 090, Loss: 2.0033, Val Acc: 0.3840, F1 (micro): 0.3840
Epoch: 100, Loss: 1.9547, Val Acc: 0.3840, F1 (micro): 0.3840
Test Accuracy: 0.3549, F1 (micro): 0.3549
Training time: 1.69 seconds
Curre

In [15]:
peak_memory_mb=f"{torch.cuda.max_memory_allocated()/1024**2:.2f}"
total_train_time=f"{end_time - start_time:.2f}"

metrics = {
    "model": "graphSAGE",
    "accuracy": test_acc,
    "f1_micro":f1_micro,
    "peak_memory_MB": peak_memory_mb,
    "train_time_sec": total_train_time,
    "mem_MB":mem_MB
}

with open("fastGCN_amazon_results.json", "w") as f:
    json.dump(metrics, f)